In [1]:
# Use the testing mode for evaluation (the whole ready-to-go model), since we 
# are not contributing to the model itself
# If user would like to use this acceleration, select the menu option 
# "Runtime" -> "Change runtime type", select "Hardware Accelerator" -> "GPU" and
# click "SAVE"
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from torchvision.models.resnet import ResNet, BasicBlock
from torchvision.datasets import MNIST
from torchvision import transforms

from tqdm.autonotebook import tqdm
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
import inspect
import time
from torch import nn, optim
import torch, os
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import Compose, ToTensor, Normalize, Resize, CenterCrop, Grayscale
from torch.utils.data import DataLoader
from PIL import Image

from random import sample

# selected ResNet34 as the model
model = torch.hub.load('pytorch/vision', 'resnet18', pretrained=True)
classes = ["Apple Golden 1", "Apple Golden 2", "Apple Golden 3", "Apple Granny Smith", "Apple Pink Lady"]

def gettrainfiles(): 
    _files, classidx, _filefullpath = [], [], []
    for cls in range(len(classes)):
        classidx += [cls] * len(os.listdir(traindir + '\\' + classes[cls]))
        _files += os.listdir(traindir + '\\' + classes[cls])
        for f in os.listdir(traindir + '\\' + classes[cls]):
            _filefullpath.append(traindir + '\\' + classes[cls] + '\\' + f)
    return [_files, classidx, _filefullpath]
def gettestfiles(): 
    _files, classidx, _filefullpath = [], [], []
    for cls in range(len(classes)):
        classidx += [cls] * len(os.listdir(testdir + '\\' + classes[cls]))
        _files += os.listdir(testdir + '\\' + classes[cls])
        for f in os.listdir(testdir + '\\' + classes[cls]):
            _filefullpath.append(testdir + '\\' + classes[cls] + '\\' + f)
    return [_files, classidx, _filefullpath]

root_path = 'C:\\Users\\wangz\\Documents\\GitHub\\ECE561MachineVision\\FinalProject\\Q2\\fruits\\fruits-360_dataset\\fruits-360'
traindir = root_path + '\\Training'
testdir = root_path + '\\Test'

trainfiles = gettrainfiles()[0]
trainclasses = gettrainfiles()[1]
trainfilefullpaths = gettrainfiles()[2]
testfiles = gettestfiles()[0]
testclasses = gettestfiles()[1]
testfilefullpaths = gettestfiles()[2]

print("Training Data Size: " + str(len(trainfiles)))
print("Testing Data Size: " + str(len(testfiles)))

C:\Users\wangz\.conda\envs\py36cv\lib\site-packages\tqdm\autonotebook.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)
Using cache found in C:\Users\wangz/.cache\torch\hub\pytorch_vision_master


Training Data Size: 2413
Testing Data Size: 805


In [2]:
# Q2 (a) recognition experiment:
# here I have not yet started fine-tuning the network 
with open(root_path + '\\' + 'imagenet_classes.txt') as f:
    labels = [line.strip() for line in f.readlines()]

model.eval()

test_true, test_pred = [], []
sample_50_image_idx = [x for x in sample(range(len(gettestfiles()[0])),50)]
input_images = sample_50_image_paths = [Image.open(gettestfiles()[2][x]) for x in sample_50_image_idx]
test_true = [classes[testclasses[x]] for x in sample_50_image_idx]
for i in range(len(input_images)):
    preprocess = transforms.Compose([
                  transforms.CenterCrop(224),
                  transforms.ToTensor(),
                  transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                       std=[0.229, 0.224, 0.225]),])
    input_tensor = preprocess(input_images[i])
    input_batch = input_tensor.unsqueeze(0)
    # create a mini-batch as expected by the model

    # move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')

    with torch.no_grad():
        output = model(input_batch)
    _, index = torch.max(output, 1)
    # The output has unnormalized scores. To get probabilities, run a 
    # softmax on it (to normalize the scores to probability).
    percentage = torch.nn.functional.softmax(output, dim=1)[0] * 100
    pred_label_i = labels[index[0]]
    test_pred.append(pred_label_i)

print("Recognition Experiment Using ImageNet pre-trained model on the Fruit360 Data:")
print('\t', "True Fruit Class", '\t', "Predicted ImageNet Class")
print('\t', "################", '\t', "########################\n")

for i in range(len(sample_50_image_idx)):
    print('\t', test_true[i], '\t\t', test_pred[i])

Recognition Experiment Using ImageNet pre-trained model on the Fruit360 Data:
	 True Fruit Class 	 Predicted ImageNet Class
	 ################ 	 ########################

	 Apple Golden 2 		 Granny Smith
	 Apple Golden 3 		 Granny Smith
	 Apple Pink Lady 		 pomegranate
	 Apple Granny Smith 		 Granny Smith
	 Apple Pink Lady 		 pomegranate
	 Apple Golden 3 		 Granny Smith
	 Apple Granny Smith 		 Granny Smith
	 Apple Golden 3 		 Granny Smith
	 Apple Pink Lady 		 pomegranate
	 Apple Golden 3 		 Granny Smith
	 Apple Golden 1 		 lemon
	 Apple Golden 3 		 Granny Smith
	 Apple Golden 3 		 Granny Smith
	 Apple Pink Lady 		 pomegranate
	 Apple Pink Lady 		 orange
	 Apple Golden 2 		 Granny Smith
	 Apple Golden 1 		 lemon
	 Apple Golden 3 		 Granny Smith
	 Apple Golden 3 		 Granny Smith
	 Apple Golden 3 		 Granny Smith
	 Apple Golden 1 		 lemon
	 Apple Golden 3 		 Granny Smith
	 Apple Golden 1 		 lemon
	 Apple Golden 2 		 Granny Smith
	 Apple Pink Lady 		 orange
	 Apple Golden 3 		 Granny Smith
	

In [30]:
# Functional Utilities
class FiveClassFruit(Dataset):
    def __init__(self, filelist, classlist, fullfilepath, transform=None):
        self.filelist = filelist
        self.classlist = classlist
        self.fullfilepath = fullfilepath
        self.transform = transform
    
    def __len__(self):
        assert len(self.filelist) == len(self.classlist) == len(self.fullfilepath)
        return len(self.filelist)
    
    def __getitem__(self, idx):
        img_name = self.filelist[idx]            
        fullname = self.fullfilepath[idx]
        cls = self.classlist[idx]
        image = Image.open(fullname)
        if self.transform:
            image = self.transform(image)
        return [image, cls]

def get_data_loaders(train_batch_size=8, val_batch_size=2):    
    data_transform = Compose([Resize((224,224)),
                              CenterCrop(224),
                              ToTensor(), 
                              Normalize(mean=[0.485, 0.456, 0.406],
                                        std=[0.229, 0.224, 0.225])])

    train_loader = DataLoader(FiveClassFruit(trainfiles, 
                                             trainclasses,
                                             trainfilefullpaths, 
                                             transform=data_transform),
                              batch_size=train_batch_size, shuffle=True)

    val_loader = DataLoader(FiveClassFruit(testfiles, 
                                           testclasses,
                                           testfilefullpaths, 
                                           transform=data_transform),
                            batch_size=val_batch_size, shuffle=False)
    # use the train dataset for both train and validation (no labels on testset)
    return train_loader, val_loader

def calculate_metric(metric_fn, true_y, pred_y):
    # multi class problems need to have averaging method
    if "average" in inspect.getfullargspec(metric_fn).args:
        return metric_fn(true_y, pred_y, average="macro")
    else:
        return metric_fn(true_y, pred_y)
    
def print_scores(p, r, f1, a, batch_size):
    for name, scores in zip(("precision", "recall", "F1", "accuracy"), (p, r, f1, a)):
        print(f"\t{name.rjust(14, ' ')}: {sum(scores)/batch_size:.4f}")

def train_and_val(model):
    batches = len(train_loader)
    val_batches = len(val_loader)

    # loop for every epoch (training + evaluation)
    for epoch in range(epochs):
        total_loss = 0

        # progress bar
        progress = tqdm(enumerate(train_loader), desc="Loss: ", total=batches)

        # ----------------- TRAINING  -------------------- 
        # set model to training
        model.train()
        for i, data in progress:
            X, y = data[0].to(device), data[1].to(device)

            # training step for single batch
            model.zero_grad()
            outputs = model(X)
            loss = loss_function(outputs, y)
            loss.backward()
            optimizer.step()

            # getting training quality data
            current_loss = loss.item()
            total_loss += current_loss

            # updating progress bar
            progress.set_description("Loss: {:.4f}".format(total_loss/(i+1)))

        # releasing unceseccary memory in GPU
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        # ----------------- VALIDATION  ----------------- 
        val_losses = 0
        precision, recall, f1, accuracy = [], [], [], []
        true_cls, pred_cls = [], []
        # set model to evaluating (testing)
        model.eval()
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                X, y = data[0].to(device), data[1].to(device)
                # Get's the prediction (outputs) from the network
                outputs = model(X)
                val_losses += loss_function(outputs, y)

                predicted_classes = torch.max(outputs, 1)[1] # get class from network's prediction
                true_cls.extend(y.cpu())
                pred_cls.extend(predicted_classes.cpu())

                # calculate P/R/F1/A metrics for batch
                for acc, metric in zip((precision, recall, f1, accuracy), 
                                       (precision_score, recall_score, f1_score, accuracy_score)):
                    acc.append(calculate_metric(metric, 
                                                y.cpu(), predicted_classes.cpu()))
            print(confusion_matrix(true_cls, pred_cls, labels=[0,1,2,3,4]))
            print(f"Epoch {epoch+1}/{epochs}, training loss: {total_loss/batches}, validation loss: {val_losses/val_batches}")
            print_scores(precision, recall, f1, accuracy, val_batches)
            losses.append(total_loss/batches) # for plotting learning curve
    print(f"Training time: {time.time()-start_ts}s")

In [32]:
# Use ResNet18 for fine-tuning fully connected layer to achieve the best results
model = torch.hub.load('pytorch/vision', 'resnet18', pretrained=True)
# Fine-tuning: First modify the model configurations
# set all weights to be fixed: not back-propogating gradients
for params in model.parameters():
    params.requires_grad = False

# set the fully connected layer to be trainable: require back-propogation of gradients
for params in model.fc.parameters():
    params.requires_grad = True

# set the number of classes as 5 (required by the project)
# replace the last layer with our custmozied linear layer with 5 out features (5 classes)
class_num = len(classes)
channel_in = model.fc.in_features
model.fc = nn.Linear(in_features=channel_in, out_features=class_num)

epochs = 3
train_loader, val_loader = get_data_loaders(64,16)# put your data loader here, play with batch size to satisfy cuda
loss_function = nn.CrossEntropyLoss() # your loss function, cross entropy works well for multi-class problems

# construct the stochastic gradient descent opimizer 
# and filter-out those parameters requiring gradient inputs (previsou layers)
optimizer = torch.optim.SGD(params=filter(lambda p: p.requires_grad, model.parameters()), lr=0.1)

# optimizer, try Adam this time to have a taste
# optimize the full connection layer only, magic numbers from online resource
optimizer = optim.Adam(model.fc.parameters(), lr=0.001, betas=(0.9, 0.999))

os.environ['CUDA_VISIBLE_DEVICES']='0'

start_ts = time.time()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_cuda = True

torch.cuda.empty_cache()
model.cuda()

losses = []

train_and_val(model)

Using cache found in C:\Users\wangz/.cache\torch\hub\pytorch_vision_master



[[164   0   0   0   0]
 [  0 164   0   0   0]
 [  0   6 155   0   0]
 [  0   0   6 158   0]
 [  0   0   0   0 152]]
Epoch 1/3, training loss: 0.7224029741789165, validation loss: 0.32721996307373047
	     precision: 0.9314
	        recall: 0.9240
	            F1: 0.9274
	      accuracy: 0.9853


KeyboardInterrupt: 

In [33]:
# Use Alexnet for testing Dropout Regularization and Batch-Normalization
# Alexnet already uses 0.5 in three dropout layers
# Modify the classfier[6] of Alexnet to fine tune
model = torch.hub.load('pytorch/vision', 'alexnet', pretrained=True)
model.classifier[6] = nn.Linear(4096, len(classes))

# set all weights to be fixed: not back-propogating gradients
for params in model.parameters():
    params.requires_grad = False

# set the fully connected layer to be trainable: require back-propogation of gradients
for params in model.classifier[6].parameters():
    params.requires_grad = True

epochs = 3
train_loader, val_loader = get_data_loaders(64,16)# put your data loader here, play with batch size to satisfy cuda
loss_function = nn.CrossEntropyLoss() # your loss function, cross entropy works well for multi-class problems

# construct the stochastic gradient descent opimizer 
# and filter-out those parameters requiring gradient inputs (previsou layers)
optimizer = torch.optim.SGD(params=filter(lambda p: p.requires_grad, model.parameters()), lr=0.1)

os.environ['CUDA_VISIBLE_DEVICES']='0'

start_ts = time.time()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_cuda = True

torch.cuda.empty_cache()
model.cuda()

losses = []

train_and_val(model)

Using cache found in C:\Users\wangz/.cache\torch\hub\pytorch_vision_master



[[164   0   0   0   0]
 [  0 164   0   0   0]
 [  0  92  63   6   0]
 [  0   2   0 162   0]
 [  0   0   0   0 152]]
Epoch 1/3, training loss: 57.803911309493216, validation loss: 5.301993370056152
	     precision: 0.8764
	        recall: 0.8278
	            F1: 0.8357
	      accuracy: 0.8775


KeyboardInterrupt: 

In [60]:
# Use Alexnet for showing the differences (effects) of dropouts and batchnorms
model = AlexNet(num_classes=len(classes))
model = torch.hub.load('pytorch/vision', 'alexnet', pretrained=True)
# deactivate the Dropout to see if there is difference or not
model.classifier[0] = nn.Identity()

epochs = 5
train_loader, val_loader = get_data_loaders(64,16)# put your data loader here, play with batch size to satisfy cuda
loss_function = nn.CrossEntropyLoss() # your loss function, cross entropy works well for multi-class problems

# construct the stochastic gradient descent opimizer 
# and filter-out those parameters requiring gradient inputs (previsou layers)
optimizer = torch.optim.SGD(params=filter(lambda p: p.requires_grad, model.parameters()), lr=0.1)

os.environ['CUDA_VISIBLE_DEVICES']='0'

start_ts = time.time()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_cuda = True

torch.cuda.empty_cache()
model.cuda()

losses = []

train_and_val(model)

Using cache found in C:\Users\wangz/.cache\torch\hub\pytorch_vision_master


RuntimeError: CUDA out of memory. Tried to allocate 144.00 MiB (GPU 0; 4.00 GiB total capacity; 2.85 GiB already allocated; 69.90 MiB free; 17.33 MiB cached)

In [58]:
model.classifier

Sequential(
  (0): Dropout(p=0.5, inplace=False)
  (1): Linear(in_features=9216, out_features=4096, bias=True)
  (2): ReLU(inplace=True)
  (3): Dropout(p=0.5, inplace=False)
  (4): Linear(in_features=4096, out_features=4096, bias=True)
  (5): ReLU(inplace=True)
  (6): Linear(in_features=4096, out_features=5, bias=True)
)

In [21]:
for p in model.classifier[6].parameters():
    print(p.requires_grad)

True
True


In [84]:
model_2 = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
    nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
    ReLU(inplace=True),
    MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)